In [1]:
import os
import pandas as pd

In [2]:
excel_path = "mouse_data.xlsx"
output_dir = "../data"
bin_sizes  = [5, 15, 30, 60]

In [3]:
xls = pd.ExcelFile(excel_path)
act_df = pd.read_excel(xls, sheet_name = "Fem Act")
temp_df = pd.read_excel(xls, sheet_name = "Fem Temp")

for df in (act_df, temp_df):
    df["day"] = df.index // 1440
    df["minute"] = df.index % 1440
    df["estrus"] = (df["day"] % 4 == 2)

In [4]:
mouse_cols = [
    c for c in act_df.columns if c not in ("day", "minute", "estrus")
]

act_long   = act_df.melt(
    id_vars = ["day", "minute", "estrus"],
    value_vars = mouse_cols,
    var_name = "mouse",
    value_name = "activity"
)

temp_long  = temp_df.melt(
    id_vars = ["day", "minute", "estrus"],
    value_vars = mouse_cols,
    var_name = "mouse",
    value_name = "temperature"
)

merged = pd.merge(
    act_long, temp_long,
    on=["day", "minute", "estrus", "mouse"]
)

In [5]:
for BIN_SIZE in bin_sizes:
    df = merged.copy()
    
    df["bin"] = df["minute"] // BIN_SIZE
    
    hourly = (
        df
        .groupby(["day", "bin", "estrus"])
        .agg(
            activity_mean = ("activity", "mean"),
            temperature_mean = ("temperature", "mean")
        )
        .reset_index()
    )

    hourly["ratio"] = hourly["temperature_mean"] / hourly["activity_mean"]

    hourly_file = os.path.join(output_dir, f"hourly_{BIN_SIZE}min.json")
    hourly.to_json(hourly_file, orient = "records", indent = 2)
    print(f"Written {hourly_file} ({len(hourly)} records)")

    summary = (
        hourly
        .groupby(["estrus", "bin"])
        .agg(
            activity_mean = ("activity_mean", "mean"),
            activity_std = ("activity_mean", "std"),
            temperature_mean = ("temperature_mean", "mean"),
            temperature_std = ("temperature_mean", "std"),
            ratio_mean = ("ratio", "mean"),
            ratio_std = ("ratio", "std"),
            count = ("day", "count")
        )
        .reset_index()
    )

    summary_file = os.path.join(output_dir, f"summary_{BIN_SIZE}min.json")
    summary.to_json(summary_file, orient = "records", indent = 2)
    print(f"Written {summary_file} ({len(summary)} records)")

Written ../data/hourly_5min.json (4032 records)
Written ../data/summary_5min.json (576 records)
Written ../data/hourly_15min.json (1344 records)
Written ../data/summary_15min.json (192 records)
Written ../data/hourly_30min.json (672 records)
Written ../data/summary_30min.json (96 records)
Written ../data/hourly_60min.json (336 records)
Written ../data/summary_60min.json (48 records)
